# PyTorch implementation for dogs vs cats classificator

In [10]:
import os
import os.path as osp
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms

In [11]:
# data loader
class Data:
    def __init__(self, shape=(224, 224)):
        # shape of the every image, 224x224 is a standart size for VGG 
        self.shape = shape
        
        self.train_dir = None
        self.train_imgs = None
        self.train_len = None
        self.train_X = None
        self.train_y = None
        
        self.test_dir = None
        self.test_imgs = None
        self.test_len = None
        self.test_X = None

    def load_train(self, path):
        self.train_dir = path
        self.train_imgs = os.listdir(path)
        self.train_len = len(self.train_imgs)

        X = torch.zeros((self.train_len, 3, 224, 224))
        y = torch.zeros((self.train_len, 2))
        tr = transforms.ToTensor()
        
        for i, cur_im_name in enumerate(self.train_imgs):
            if (i % 1000 == 999):
                print("%.1f percents of train data loaded" % (i / self.train_len * 100))
            im_path = osp.join(self.train_dir, cur_im_name)
            im = Image.open(im_path)
            assert im.size == self.shape
            im = tr(im)
            X[i, :] = im
            if cur_im_name[:3] == "cat":
                y[i, 0] = 0
                y[i, 1] = 1
            else:
                y[i, 0] = 1
                y[i, 1] = 0
        self.train_X = X.float()
        self.train_y = y.float()
        return self.train_X, self.train_y

    def load_test(self, path):
        self.test_dir = path
        self.test_imgs = sorted(os.listdir(path)[:],  key=lambda x: int(x[:-4])) # remove .jpg extension
        self.test_len = len(self.test_imgs)

        X = torch.zeros((len(self.test_imgs), 3, 224, 224))
        tr = transforms.ToTensor()
        
        for i, cur_im_name in enumerate(self.test_imgs):
            if (i % 100 == 0):
                print("%.1f percents of test data loaded" % (i / self.test_len * 100))
            im_path = osp.join(self.test_dir, cur_im_name)
            im = Image.open(im_path)
            assert im.size == self.shape
            im = tr(im)
            X[i, :] = im
        self.test_X = X.float()
        return self.test_X

# resize images before running the net
def resize_and_save(folder_from, folder_to, shape=(224, 224)):
    imgs = os.listdir(folder_from)
    l = len(imgs)
    for n, i in enumerate(imgs):
        im = Image.open(os.path.join(folder_from, i)).resize(shape)
        if (n % 100 == 0):
            print("%.1f percents resized" % (n / l * 100))
        im.convert('RGB').save(os.path.join(folder_to, i))

In [19]:
def train(model, X, y, optimizer, criterion, CUDA=False, epo=10, batch_size=20, save_model_to=None):
    print("training...")
    for e in range(epo):
        running_loss = 0
        print("{} epoch: ".format(e))
        for batch in range(0, int(X.shape[0] / batch_size)):
            try:
                batch_X = X[batch * batch_size:(batch + 1) * batch_size]
                batch_y = y[batch * batch_size:(batch + 1) * batch_size]
            except IndexError:
                print("batch out of range")
                break
            if CUDA:
                batch_X, batch_y = batch_X.cuda(), batch_y.cuda()
            optimizer.zero_grad()
            y_pred = model(batch_X)
            loss = criterion(y_pred, torch.max(batch_y, 1)[1])
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if (batch % 1000 == 999):
                print("\tbatch: {}/{}, loss: {}".format(batch+1, int(X.shape[0] / batch_size), running_loss))
                running_loss = 0

    if save_model_to is not None:
        torch.save(model, save_model_to)
    return model

In [20]:
def run(model, X, CUDA=False, res_filename="result.csv"):
    if CUDA:
        X = X.cuda()
    sm = torch.nn.Softmax(dim=1)
    filename = res_filename

    # test one by one
    for batch in range(int(X.shape[0])):
        batch_X = X[batch:batch + 1]
        if CUDA:
            batch_X = batch_X.cuda()
        y_pred = sm(model(batch_X)).round().int().tolist()[0]
        y = [str(batch+1), "0"] if y_pred == [0, 1] else [str(batch+1), "1"]
        print(",".join(y), file=open(filename, "a+"))
        if (batch % 1000 == 999):
            print("batch: {}".format(batch+1))

In [21]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

CUDA = torch.cuda.is_available()

# create a model
print("creating a model...")
model = torchvision.models.vgg13_bn(num_classes=2)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

if CUDA:
    model = model.cuda()
    criterion = criterion.cuda()
print("model created")

creating a model...
model created


In [15]:
# load data
print("loading data...")
DATA_PATH = "data"
data = Data()
X, y = data.load_train(osp.join(DATA_PATH, "train"))
print("data loaded")
print("train size: {}".format(data.train_len))

loading data...
4.0 percents of train data loaded
8.0 percents of train data loaded
12.0 percents of train data loaded
16.0 percents of train data loaded
20.0 percents of train data loaded
24.0 percents of train data loaded
28.0 percents of train data loaded
32.0 percents of train data loaded
36.0 percents of train data loaded
40.0 percents of train data loaded
44.0 percents of train data loaded
48.0 percents of train data loaded
52.0 percents of train data loaded
56.0 percents of train data loaded
60.0 percents of train data loaded
64.0 percents of train data loaded
68.0 percents of train data loaded
72.0 percents of train data loaded
76.0 percents of train data loaded
80.0 percents of train data loaded
84.0 percents of train data loaded
88.0 percents of train data loaded
92.0 percents of train data loaded
96.0 percents of train data loaded
100.0 percents of train data loaded
data loaded
train size: 25000


In [22]:
model = train(model, X, y, optimizer, criterion, CUDA, epo = 1, batch_size = 5)
# train(model, X, y, optimizer, criterion, CUDA=False, epo = 10, batch_size = 20, save_model_to=None
data = Data()

training...
0 epoch: 
	batch: 1000/5000, loss: 1154.607865867205
	batch: 2000/5000, loss: 704.5586712956429
	batch: 3000/5000, loss: 694.5664251744747
	batch: 4000/5000, loss: 692.1011261343956
	batch: 5000/5000, loss: 677.0551397204399


In [23]:
X = data.load_test("data/test1")
run(model, X, CUDA, "results.csv")

0.0 percents of test data loaded
0.8 percents of test data loaded
1.6 percents of test data loaded
2.4 percents of test data loaded
3.2 percents of test data loaded
4.0 percents of test data loaded
4.8 percents of test data loaded
5.6 percents of test data loaded
6.4 percents of test data loaded
7.2 percents of test data loaded
8.0 percents of test data loaded
8.8 percents of test data loaded
9.6 percents of test data loaded
10.4 percents of test data loaded
11.2 percents of test data loaded
12.0 percents of test data loaded
12.8 percents of test data loaded
13.6 percents of test data loaded
14.4 percents of test data loaded
15.2 percents of test data loaded
16.0 percents of test data loaded
16.8 percents of test data loaded
17.6 percents of test data loaded
18.4 percents of test data loaded
19.2 percents of test data loaded
20.0 percents of test data loaded
20.8 percents of test data loaded
21.6 percents of test data loaded
22.4 percents of test data loaded
23.2 percents of test data 